In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import seaborn as sns
#import statsmodels.api as sm
import matplotlib.pyplot as plt
from skopt import BayesSearchCV
from skopt.space import Real


In [ ]:
# wrapper around train_test_split for convenience
def traintestsplit(df, random_state=0):
    X = df.drop(['cc_pixel_intensity_ratio', 'line_id'],axis=1)
    Y = df['cc_pixel_intensity_ratio']
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=random_state)
    return X_train, X_test, Y_train, Y_test

In [ ]:
# this cell loads dataframes and keeps only features of interest
# each dataframe has the naming convention
# group_type_df where type may be networked or standalones (i.e. non-networked)
# we divide the last four groups into net and standalones in a different cell below

# Note: Replace * with actual file path
# Name of the Dataframe as per name of the experimental group
# format: groupname_(net or standalone)_df
tol1hr_standalones_df = pd.read_csv('*/tol1hr_standalones.csv', index_col=None)
tol1hr_standalones_df.drop(labels=tol1hr_standalones_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_standalones', 'edge_density', 'node_density']), axis=1, inplace=True)

tol1hr_net_df = pd.read_csv('*/tol1hr_net.csv', index_col=None)
tol1hr_net_df.drop(labels=tol1hr_net_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_networks', 'edge_density', 'node_density']), axis=1, inplace=True)

tol4_standalones_df = pd.read_csv('*/tol4_standalones.csv', index_col=None)
tol4_standalones_df.drop(labels=tol4_standalones_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_standalones', 'edge_density', 'node_density']), axis=1, inplace=True)

tol4_net_df = pd.read_csv('*/tol4_net.csv', index_col=None)
tol4_net_df.drop(labels=tol4_net_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_networks', 'edge_density', 'node_density']), axis=1, inplace=True)

TCDD10nM1_standalones_df = pd.read_csv('*/TCDD10nM1_standalones.csv', index_col=None)
TCDD10nM1_standalones_df.drop(labels=TCDD10nM1_standalones_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_standalones', 'edge_density', 'node_density']), axis=1, inplace=True)

TCDD10nM1_net_df = pd.read_csv('*/TCDD10nM1_net.csv', index_col=None)
TCDD10nM1_net_df.drop(labels=TCDD10nM1_net_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_networks', 'edge_density', 'node_density']), axis=1, inplace=True)

TCDD10nM4_standalones_df = pd.read_csv('*/TCDD10nM4_standalones.csv', index_col=None)
TCDD10nM4_standalones_df.drop(labels=TCDD10nM4_standalones_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_standalones', 'edge_density', 'node_density']), axis=1, inplace=True)

TCDD10nM4_net_df = pd.read_csv('*/TCDD10nM4_net.csv', index_col=None)
TCDD10nM4_net_df.drop(labels=TCDD10nM4_net_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_networks', 'edge_density', 'node_density']), axis=1, inplace=True)

TCDD1nM1_standalones_df = pd.read_csv('*/TCDD1nM1_standalones.csv', index_col=None)
TCDD1nM1_standalones_df.drop(labels=TCDD1nM1_standalones_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_standalones', 'edge_density', 'node_density']), axis=1, inplace=True)

TCDD1nM1_net_df = pd.read_csv('*/TCDD1nM1_net.csv', index_col=None)
TCDD1nM1_net_df.drop(labels=TCDD1nM1_net_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_networks', 'edge_density', 'node_density']), axis=1, inplace=True)

TCDD1nM4_standalones_df = pd.read_csv('*/TCDD1nM4_standalones.csv', index_col=None)
TCDD1nM4_standalones_df.drop(labels=TCDD1nM4_standalones_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_standalones', 'edge_density', 'node_density']), axis=1, inplace=True)

TCDD1nM4_net_df = pd.read_csv('*/TCDD1nM4_net.csv', index_col=None)
TCDD1nM4_net_df.drop(labels=TCDD1nM4_net_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_networks', 'edge_density', 'node_density']), axis=1, inplace=True)

TO_df = pd.read_csv('*/TO_sheet.csv', index_col=None)
TO_df.drop(labels=TO_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_networks', 'normalized_length_by_standalones', 'edge_density', 'node_density']), axis=1, inplace=True)

_1n_df = pd.read_csv('*/1n_sheet.csv', index_col=None)
_1n_df.drop(labels=_1n_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_networks','normalized_length_by_standalones', 'edge_density', 'node_density']), axis=1, inplace=True)

_10_df = pd.read_csv('*/10_sheet.csv', index_col=None)
_10_df.drop(labels=_10_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_networks', 'normalized_length_by_standalones', 'edge_density', 'node_density']), axis=1, inplace=True)

DS_df = pd.read_csv('*/DS_sheet.csv', index_col=None)
DS_df.drop(labels=DS_df.columns.difference(['cc_length_(um)', 'nodes', 'edges', 'cc_pixel_intensity_ratio', 'cc_max_PK', 'line_id', 'diameter', 'element_length_(um)', 'normalized_length_by_networks', 'normalized_length_by_standalones', 'edge_density', 'node_density']), axis=1, inplace=True)

In [ ]:
# we drop duplicates based on the element_length column
tol4_net_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
tol1hr_net_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
TCDD10nM1_net_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
TCDD10nM4_net_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
TCDD1nM1_net_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
TCDD1nM4_net_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
tol4_standalones_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
tol1hr_standalones_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
TCDD10nM1_standalones_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
TCDD10nM4_standalones_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
TCDD1nM1_standalones_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
TCDD1nM4_standalones_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
TO_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
_1n_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
_10_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)
DS_df.drop_duplicates(subset=['element_length_(um)'], inplace=True, ignore_index=True)

In [ ]:
# splitting the last four cells into net and standalones
# if there's only two nodes and one edge, that's a standalone
# Note: This step depends on how your data is grouped, if all data is grouped together, the following code needs to be
# executed for all experimental groups

TO_standalones_df = TO_df[((TO_df['nodes'] == 2) & (TO_df['edges'] == 1))].copy()
TO_standalones_df.drop(labels=['normalized_length_by_networks'], axis=1, inplace=True)
TO_net_df = TO_df[~((TO_df['nodes'] == 2) & (TO_df['edges'] == 1))].copy()
TO_net_df.drop(labels=['normalized_length_by_standalones'], axis=1, inplace=True)

_1n_standalones_df = _1n_df[((_1n_df['nodes'] == 2) & (_1n_df['edges'] == 1))].copy()
_1n_standalones_df.drop(labels=['normalized_length_by_networks'], axis=1, inplace=True)
_1n_net_df = _1n_df[~((_1n_df['nodes'] == 2) & (_1n_df['edges'] == 1))].copy()
_1n_net_df.drop(labels=['normalized_length_by_standalones'], axis=1, inplace=True)

_10_standalones_df = _10_df[((_10_df['nodes'] == 2) & (_10_df['edges'] == 1))].copy()
_10_standalones_df.drop(labels=['normalized_length_by_networks'], axis=1, inplace=True)
_10_net_df = _10_df[~((_10_df['nodes'] == 2) & (_10_df['edges'] == 1))].copy()
_10_net_df.drop(labels=['normalized_length_by_standalones'], axis=1, inplace=True)

DS_standalones_df = DS_df[((DS_df['nodes'] == 2) & (DS_df['edges'] == 1))].copy()
DS_standalones_df.drop(labels=['normalized_length_by_networks'], axis=1, inplace=True)
DS_net_df = DS_df[~((DS_df['nodes'] == 2) & (DS_df['edges'] == 1))].copy()
DS_net_df.drop(labels=['normalized_length_by_standalones'], axis=1, inplace=True)

In [ ]:
# we create the pooled datasets for net and standalones
standalones_pooled_df = pd.concat([tol1hr_standalones_df, tol4_standalones_df, TCDD10nM1_standalones_df, TCDD10nM4_standalones_df, TCDD1nM1_standalones_df, TCDD1nM4_standalones_df, TO_standalones_df, _1n_standalones_df, _10_standalones_df, DS_standalones_df], axis=0, ignore_index=True)
net_pooled_df = pd.concat([tol1hr_net_df, tol4_net_df, TCDD10nM1_net_df, TCDD10nM4_net_df, TCDD1nM1_net_df, TCDD1nM4_net_df, TO_net_df, _1n_net_df, _10_net_df, DS_net_df], axis=0, ignore_index=True)

In [ ]:
# we apply a log-transformation for all length related features
net_pooled_df['normalized_length_by_networks'] = np.log(net_pooled_df['normalized_length_by_networks'])
standalones_pooled_df['normalized_length_by_standalones'] = np.log(standalones_pooled_df['normalized_length_by_standalones'])
net_pooled_df['element_length_(um)'] = np.log(net_pooled_df['element_length_(um)'])
standalones_pooled_df['element_length_(um)'] = np.log(standalones_pooled_df['element_length_(um)'])
net_pooled_df['cc_length_(um)'] = np.log(net_pooled_df['cc_length_(um)'])
standalones_pooled_df['cc_length_(um)'] = np.log(standalones_pooled_df['cc_length_(um)'])

In [ ]:
# drops nans (artifact of log transform/naturally occuring) from data
net_pooled_df.replace([np.inf, -np.inf], np.nan, inplace=True)
net_pooled_df.dropna(how="all", inplace=True)

standalones_pooled_df.replace([np.inf, -np.inf], np.nan, inplace=True)
standalones_pooled_df.dropna(how="all", inplace=True)

In [ ]:
# we filter out the outliers
net_pooled_df = net_pooled_df[net_pooled_df['cc_pixel_intensity_ratio'] <= 0.5]
standalones_pooled_df = standalones_pooled_df[standalones_pooled_df['cc_pixel_intensity_ratio'] <= 0.5]

### Training Models

In [ ]:
# we select model hyperparams using the skopt library
# read docs here: https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html
# for this magnitude of hyperparameter values, there are only marginal differences for parameters that are tuned slightly differently
# so i've only included one example of this function call here
# its also the most time consuming function to call in the entire pipeline
X_train, X_test, y_train, y_test = traintestsplit(net_pooled_df)

net_pooled_opt = BayesSearchCV(
    RandomForestRegressor(),
    {
        'n_estimators': (70,100),
        'max_depth': (30,50),
        'ccp_alpha': Real(0,0.5),
    },
    n_iter = 50
)

net_pooled_opt.fit(X_train, y_train)
print(net_pooled_opt.score(X_test, y_test))
print(net_pooled_opt.best_params_)

In [ ]:
# wrapper around 5-fold CV and fitting random forests

def evaluate_model(data, model):
    X_train, X_test, y_train, y_test = traintestsplit(
        data, random_state = 0
    )

    cv_scores = cross_val_score(
        model, X_train, y_train,
        cv=5, scoring='r2'
    )

    model.fit(X_train, y_train)

    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)

    return {
        'cv_scores': cv_scores,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'train_score': train_score,
        'test_score': test_score
    }

The next few cells run the function above on the various dataframes and are segregated by type

In [ ]:
print("standalones_pooled_df")
standalones_pooled_rf = RandomForestRegressor(max_depth=35,n_estimators=75, criterion='squared_error')
results = evaluate_model(standalones_pooled_df, standalones_pooled_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("net_pooled_df")
net_pooled_rf = RandomForestRegressor(max_depth=35,n_estimators=75, criterion='squared_error')
results = evaluate_model(net_pooled_df, net_pooled_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

In [ ]:
print("TCDD1nM4_net_df")
TCDD1nM4_rf = RandomForestRegressor(max_depth=44,n_estimators=105, criterion='squared_error')
results = evaluate_model(TCDD1nM4_net_df, TCDD1nM4_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("TCDD1nM1_net_df")
TCDD1nM1_rf = RandomForestRegressor(max_depth=43,n_estimators=105, criterion='squared_error')
results = evaluate_model(TCDD1nM1_net_df, TCDD1nM1_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("TCDD10nM4_net_df")
TCDD10nM4_rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(TCDD10nM4_net_df, TCDD10nM4_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("TCDD10nM1_net_df")
TCDD10nM1_rf = RandomForestRegressor(max_depth=45,n_estimators=98, criterion='squared_error')
results = evaluate_model(TCDD1nM1_net_df, TCDD1nM1_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("tol4_net_df")
tol4_rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(tol4_net_df, tol4_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("tol1hr_net_df")
tol1hr_rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(tol1hr_net_df, tol1hr_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

In [ ]:
print("DS_net_df")
DS_net_rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(DS_net_df, DS_net_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("TO__net_df")
TO__net_rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(TO_net_df, TO__net_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("_1n_net_df")
_1n_net_rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(_1n_net_df, _1n_net_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("_10_net_df")
_10_net_rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(_10_net_df, _10_net_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

In [ ]:
print("TCDD1nM4_standalones_df")
TCDD1nM4_rf = RandomForestRegressor(max_depth=44,n_estimators=105, criterion='squared_error')
results = evaluate_model(TCDD1nM4_standalones_df, TCDD1nM4_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("TCDD1nM1_standalones_df")
TCDD1nM1_rf = RandomForestRegressor(max_depth=43,n_estimators=105, criterion='squared_error')
results = evaluate_model(TCDD1nM1_standalones_df, TCDD1nM1_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("TCDD10nM4_standalones_df")
TCDD10nM4_rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(TCDD10nM4_standalones_df, TCDD10nM4_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("TCDD10nM1_standalones_df")
TCDD10nM1_rf = RandomForestRegressor(max_depth=45,n_estimators=98, criterion='squared_error')
results = evaluate_model(TCDD1nM1_standalones_df, TCDD1nM1_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("tol4_standalones_df")
tol4_rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(tol4_standalones_df, tol4_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("tol1hr_standalones_df")
tol1hr_rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(tol1hr_standalones_df, tol1hr_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

In [ ]:
print("DS_standalones_df")
DS_rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(DS_standalones_df, DS_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("TO_standalones_df")
TO__rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(TO_standalones_df, TO__rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("_1n_standalones_df")
_1n_rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(_1n_standalones_df, _1n_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

print("_10_standalones_df")
_10_rf = RandomForestRegressor(max_depth=45,n_estimators=96, criterion='squared_error')
results = evaluate_model(_10_standalones_df, _10_rf)
print("Cross-validation scores:", results['cv_scores'])
print(f"CV mean: {results['cv_mean']:.3f} (+/- {results['cv_std']*2:.3f})")
print(f"Training score: {results['train_score']:.3f}")

print(f"Test score: {results['test_score']:.3f}")

### Feature Importance

In [ ]:
X_train, X_test, y_train, y_test = traintestsplit(net_pooled_df)
importances = net_pooled_rf.feature_importances_
indices = np.argsort(importances)

fig, ax = plt.subplots()
ax.barh(range(len(importances)), importances[indices])
ax.set_yticks(range(len(importances)))
_ = ax.set_yticklabels(np.array(X_train.columns)[indices])

In [ ]:
X_train, X_test, y_train, y_test = traintestsplit(standalones_pooled_df)
importances = standalones_pooled_rf.feature_importances_
indices = np.argsort(importances)

fig, ax = plt.subplots()
ax.barh(range(len(importances)), importances[indices])
ax.set_yticks(range(len(importances)))
_ = ax.set_yticklabels(np.array(X_train.columns)[indices])

### Dataset Size Reduction Experiment

Can we attribute better performance on net_pooled_df to the fact that it simply has more rows of data? The experiment below gradually reduces the number of rows in net_pooled_df till it matches the size of standalones_pooled_df and plots the reduction in $R^2$

In [ ]:
def reduce_and_score(df, target_size, step=3000):
    scores = []
    sizes = []
    while len(df) > target_size:
        df = df.sample(frac=1).iloc[:-step].reset_index(drop=True)
        sizes.append(len(df))

        X_train, X_test, Y_train, Y_test = traintestsplit(df)
        rf = RandomForestRegressor(max_depth=45, n_estimators=96, criterion='squared_error')
        rf.fit(X_train, Y_train)
        scores.append(rf.score(X_test, Y_test))
    return sizes, scores

target_size = len(standalones_pooled_df)
sizes, scores = reduce_and_score(net_pooled_df, target_size)

# we also train on standalones to get a reference point
X_train_sa, X_test_sa, Y_train_sa, Y_test_sa = traintestsplit(standalones_pooled_df)
rf_sa = RandomForestRegressor(max_depth=45, n_estimators=96, criterion='squared_error')
rf_sa.fit(X_train_sa, Y_train_sa)
standalone_score = rf_sa.score(X_test_sa, Y_test_sa)

plt.figure(figsize=(10, 6))

plt.plot(sizes[::-1], scores[::-1], marker='o', label='Reduced Dataset')

plt.scatter(target_size, standalone_score, color='red', s=100, marker='*',
            label='Standalone Model')

plt.xlabel('Dataset Size')
plt.gca().invert_xaxis()
plt.ylabel('R² Score')
plt.title('R² Score vs. Dataset Size (Random Removal)')
plt.grid(True)
plt.ylim(0, 1)
plt.legend()
plt.show()

### Plots of Error Distribution and Target Feature Distribution

In [ ]:
def regression_dot_plot(y_pred, y_test, color):
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, s=18, c=color, alpha=0.8)

    max_val = max(np.max(y_test), np.max(y_pred))
    plt.plot([0, max_val], [0, max_val], 'k--', alpha=0.8)

    plt.xlabel('True Values', fontsize=24, labelpad=15)
    plt.ylabel('Predictions', fontsize=24, labelpad=15)
    plt.title('Regression Dot Plot', fontsize=24, pad=20)
    plt.tick_params(axis='both', which='major', labelsize=24)

    plt.tight_layout()
    plt.show()

In [ ]:
def error_versus_frequency_plot(y_pred, Y_test, color):
    errors = np.abs(y_pred - Y_test)

    num_bins = int((1/4)*np.sqrt(len(Y_test)))
    bins = np.linspace(Y_test.min(), Y_test.max(), num_bins)
    bin_indices = np.digitize(Y_test, bins)

    bin_errors = [errors[bin_indices == i] for i in range(1, num_bins)]
    avg_errors = [np.mean(be) if len(be) > 0 else 0 for be in bin_errors]

    bin_centers = (bins[:-1] + bins[1:]) / 2

    bin_counts = [np.sum(bin_indices == i) for i in range(1, num_bins)]

    max_count = max(bin_counts) if bin_counts else 1
    max_error = max(avg_errors) if avg_errors else 1
    normalized_counts = [count * max_error / max_count for count in bin_counts]

    plt.figure(figsize=(10, 6))

    plt.bar(bin_centers, avg_errors, width=(bins[1] - bins[0]) * 0.9,
            edgecolor=color, linewidth=2, fill=False,
            label='Mean Absolute Error')

    plt.plot(bin_centers, normalized_counts, color=color, linewidth=3,
             marker='o', markersize=8, label='Target Value Distribution (scaled)')

    plt.tick_params(axis='both', which='major', labelsize=24)

    plt.xlabel('Binned Target Value', labelpad=15)
    plt.ylabel('Mean Absolute Prediction Error', labelpad=15)
    plt.title('Mean Absolute Error and Target Value Distribution')
    plt.legend()

    ax2 = plt.gca().twinx()
    ax2.set_ylabel('Percentage of Test Set', labelpad=15)
    ax2.set_ylim(0, 25)

    scaling_factor = max_error / (25 * max_count / len(Y_test))
    ax2.plot(bin_centers, [count * scaling_factor for count in bin_counts], alpha=0)
    ax2.tick_params(axis='y', labelsize=24, pad=20)

    plt.gca().tick_params(axis='both', pad=20)
    plt.tight_layout()
    plt.show()

In [ ]:
X_train, X_test, y_train, y_test = traintestsplit(
        net_pooled_df
    )
y_pred = net_pooled_rf.predict(X_test)
print(len(X_test))
print(net_pooled_rf.score(X_test, y_test))
regression_dot_plot(y_pred, y_test, '#CC0000')
error_versus_frequency_plot(y_pred, y_test, '#CC0000')

X_train, X_test, y_train, y_test = traintestsplit(
        standalones_pooled_df
    )
y_pred = standalones_pooled_rf.predict(X_test)
print(len(X_test))
print(standalones_pooled_rf.score(X_test, y_test))
regression_dot_plot(y_pred, y_test, 'black')
error_versus_frequency_plot(y_pred, y_test, 'black')